<img src="https://swan.web.cern.ch/sites/swan.web.cern.ch/files/pictures/logo_swan_letters.png" alt="SWAN" style="float: left; width: 20%; margin-right: 15%; margin-left: 15%; margin-bottom: 2.0em;">
<img src="https://spark.apache.org/images/spark-logo-trademark.png" alt="EP-SFT" style="float: left; width: 25%; margin-right: 15%; margin-bottom: 2.0em;">
<p style="clear: both;">
<div style="text-align:center"><h1>Physics analysis with Apache Spark using Coffea and Laurelin packages from Fermilab</h1></div>
<div style="text-align:center"><i>Author: Lindsey Gray; Contact: Lindsey Gray / Prasanth Kothuri</i></div>
<hr style="border-top-width: 4px; border-top-color: #34609b;">

# Dimuon spectrum

This code is a columnar adaptation of [a ROOT tutorial](https://root.cern.ch/doc/master/df102__NanoAODDimuonAnalysis_8py.html) showcasing the awkward array toolset, and utilizing Coffea [histograms](https://coffeateam.github.io/coffea/modules/coffea.hist.html).
This also shows the analysis object syntax implemented by Coffea [JaggedCandidateArray](https://coffeateam.github.io/coffea/api/coffea.analysis_objects.JaggedCandidateMethods.html), and the usage of custom [accumulators](https://coffeateam.github.io/coffea/api/coffea.processor.AccumulatorABC.html) other than histograms.  Further, it introduces the [processor](https://coffeateam.github.io/coffea/api/coffea.processor.ProcessorABC.html) concept and the interface to apache spark.

# Configuration

In [1]:
LOCAL_EXECUTION_MODE=True
LOCAL_FILE_MODE=False

# Payload
Now let's make some Coffea

In [2]:
#
# All of these configs are either:
#   * Coffea specific and rolled into the library
#   * Site-specific and rolled into site-wide Spark configs
# The goal is that a user never sees any of this
#

import pyspark
import os, os.path

# Work around arrow compatibility issue
os.environ['ARROW_PRE_0_15_IPC_FORMAT'] = "1"

if LOCAL_EXECUTION_MODE:
    masterURL = "local[16]"
else:
    masterURL = "spark://hammer-c026:7077"

spark = pyspark.sql.SparkSession.builder \
    .master(masterURL) \
    .config('spark.jars.packages',
            'edu.vanderbilt.accre:laurelin:1.0.2') \
    .config('spark.driver.memory', '24g') \
    .config('spark.executor.memory', '24g') \
    .config('spark.sql.execution.arrow.enabled', 'false') \
    .config('spark.speculation', 'true') \
    .config('spark.sql.execution.arrow.maxRecordsPerBatch', 200000) \
    .config('spark.driver.extraClassPath', os.path.join(os.getcwd(), '..', 'hadoop-xrootd-1.0.4-jar-with-dependencies.jar')) \
    .config('spark.executor.extraClassPath', os.path.join(os.getcwd(), '..', 'hadoop-xrootd-1.0.4-jar-with-dependencies.jar')) \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .getOrCreate()

print(spark)

In [3]:
import time

%matplotlib inline
from coffea import hist
from coffea.analysis_objects import JaggedCandidateArray
import coffea.processor as processor

In [ ]:
'''
# Look at ProcessorABC documentation to see the expected methods and what they are supposed to do
class DimuonProcessor(processor.ProcessorABC):
    def __init__(self):
        self._columns = ['nMuon', 'Muon_pt', 'Muon_eta', 'Muon_phi', 'Muon_mass', 'Muon_charge']
        dataset_axis = hist.Cat("dataset", "Primary dataset")
        mass_axis = hist.Bin("mass", r"$m_{\mu\mu}$ [GeV]", 30000, 0.25, 300)
        
        self._accumulator = processor.dict_accumulator({
            'mass': hist.Hist("Counts", dataset_axis, mass_axis),
            'cutflow': processor.defaultdict_accumulator(int),
        })
    
    @property
    def accumulator(self):
        return self._accumulator
    
    @property
    def columns(self):
        return self._columns
    
    def process(self, df):
        output = self.accumulator.identity()
        
        dataset = df['dataset']
        muons = JaggedCandidateArray.candidatesfromcounts(
            df['nMuon'],
            pt=df['Muon_pt'].content,
            eta=df['Muon_eta'].content,
            phi=df['Muon_phi'].content,
            mass=df['Muon_mass'].content,
            charge=df['Muon_charge'].content,
            )
        
        output['cutflow']['all events'] += muons.size
        
        twomuons = (muons.counts == 2)
        output['cutflow']['two muons'] += twomuons.sum()
        
        opposite_charge = twomuons & (muons['charge'].prod() == -1)
        output['cutflow']['opposite charge'] += opposite_charge.sum()
        
        dimuons = muons[opposite_charge].distincts()
        output['mass'].fill(dataset=dataset, mass=dimuons.mass.flatten())
        
        return output

    def postprocess(self, accumulator):
        return accumulator
'''

Prepare data samples
===

In [4]:
from python.samples_info import SamplesInfo
samples = [
### Data ###
    'data_A', #not available for 2016
    'data_B',
#    'data_C',
#    'data_D','data_E',
#    'data_F',
#    'data_G','data_H',

### Essential MC ###    
#    'dy_m105_160_amc', 
#    'dy_m105_160_vbf_amc',
#      'ggh_amcPS', 
#     'vbf_powhegPS', 
#     'ttjets_dl',
#    "ewk_lljj_mll105_160_ptj0",

### Non-essential MC ### 
#     'ttjets_sl',
#     'ttz',
#     'ttw',
#     'st_tw_top','st_tw_antitop',
#     'ww_2l2nu',
#     'wz_2l2q',
#     'wz_3lnu',
#     'wz_1l1nu2q',
#      'zz',
# # ##
    
]

purdue = 'root://xrootd.rcac.purdue.edu/'

year = '2017'
label = 'test' # change this to save to other directory

samp_info = SamplesInfo(year=year, out_path=f'test_{year}_{label}', server=purdue, debug=True)

# 'outer' refers to parallelization by sample, 'inner' - by ROOT file in each sample
samp_info.load(samples, nchunks=1, parallelize_outer=1, parallelize_inner=10)
samp_info.compute_lumi_weights()


year:  2017
Default lumi:  41530.0
Loading data_A
Couldn't load data_A! Skipping.
Loading data_B

Total events in 2017: 769080716
Loaded 144492 of 2017 data events
This is ~ 0.02% of 2017 data.
Integrated luminosity: 41530.0/pb

Missing samples: ['data_A']
Loading took 15.61 s


In [5]:
import pyspark.sql
from pyarrow.compat import guid
from coffea.processor.spark.detail import _spark_initialize, _spark_stop
from coffea.processor.spark.spark_executor import spark_executor
from python.dimuon_processor import DimuonProcessor
spark.conf.set("spark.sql.execution.arrow.enabled", "false")
print("PWD IS %s " % os.getcwd())

partitionsize = 20000000
thread_workers = 16

tstart = time.time()    
if LOCAL_FILE_MODE == False:
    file_prefix = 'root://eospublic.cern.ch//eos/root-eos/cms_opendata_2012_nanoaod/'
else:
    file_prefix = '../input/'
    
fileset = {
    'DoubleMuon': { 'files': [
        file_prefix + 'Run2012B_DoubleMuParked.root',
        file_prefix + 'Run2012C_DoubleMuParked.root',
                             ], 
                    'treename': 'Events'
                  }
}

output = processor.run_spark_job(fileset, DimuonProcessor(samp_info=samp_info, do_jecunc=False, use_spark=True), spark_executor, 
                                 spark=spark, partitionsize=partitionsize, thread_workers=thread_workers,
                                 executor_args={'file_type': 'edu.vanderbilt.accre.laurelin.Root', 'cache': False}
                                )

elapsed = time.time() - tstart
print(output)

PWD IS /home/spiperov/tmp/Jun5_SPARK_tests/hammer-slurm-spark-coffea/notebooks 


Py4JJavaError: An error occurred while calling o527.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 673, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/spiperov/.conda/envs/cent7/5.3.1-py37/hmumu_coffea/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/home/spiperov/.conda/envs/cent7/5.3.1-py37/hmumu_coffea/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/spiperov/.conda/envs/cent7/5.3.1-py37/hmumu_coffea/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 290, in dump_stream
    for series in iterator:
  File "<string>", line 1, in <lambda>
  File "/hom

In [ ]:
ax = hist.plot1d(output['mass'], overlay='dataset')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylim(0.1, 1e6)

import matplotlib.pyplot as plt
ax.figure.set_size_inches(10,10)
txt_opts = {'horizontalalignment': 'center',
            'verticalalignment': 'center',
            'transform': ax.transAxes}
plt.text(0.85, 0.75, 'Z', **txt_opts)
plt.text(0.55, 0.77, r"$\Upsilon$(1,2,3S)", **txt_opts)
plt.text(0.37, 0.95, r"J/$\Psi$", **txt_opts)
plt.text(0.40, 0.77, r"$\Psi$'", **txt_opts)
plt.text(0.22, 0.80, r"$\phi$", **txt_opts)
plt.text(0.16, 0.83, r"$\rho,\omega$", **txt_opts)
plt.text(0.11, 0.78, r"$\eta$", **txt_opts);

In [ ]:
print("Events/s:", output['cutflow']['all events']/elapsed)